In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

In [6]:
df = pd.read_csv("/Users/islandchen/Documents/学习/搜推/recommend_and_search/Search/dataset/seattle-weather.csv")
df.isnull().sum()

date             0
precipitation    0
temp_max         0
temp_min         0
wind             0
weather          0
dtype: int64

In [7]:
df.duplicated().sum()

np.int64(0)

In [8]:
training_set = df.iloc[:,2].values
training_set

array([12.8, 10.6, 11.7, ...,  7.2,  5.6,  5.6])

In [9]:
def train_to_xy(training_set,window_size=10):
    x_train = []
    y_train = []
    
    for i in range(10,len(training_set)):
        x_train.append(training_set[i-10:i])
        y_train.append(training_set[i])
    x_train,y_train = np.array(x_train),np.array(y_train)
    return x_train,y_train

In [10]:
X,Y = train_to_xy(training_set)
print(len(X),len(Y))

1451 1451


In [11]:
x_train = X[:800]
y_train = Y[:800]
x_valid = X[800:1000]
y_valid = Y[800:1000]
x_test = X[1000:]
y_test = Y[1000:]
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
x_valid = np.reshape(x_valid,(x_valid.shape[0],x_valid.shape[1],1))
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

In [12]:
class RNNmodel(nn.Module):
    def __init__(self,input_size,hidden_size,n_layers,batch_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.bs = batch_size
        self.rnn = nn.RNN(input_size,hidden_size,nonlinearity = 'relu',batch_first=True)
        self.fc = nn.Linear(hidden_size,1)
        
        self.h0 = torch.zeros(n_layers,batch_size,hidden_size)
    
    def forward(self,x):
        out,h_n = self.rnn(x,self.h0)
        h_n = h_n.view(self.bs,self.hidden_size)
        h_n = self.fc(h_n)
        return out,h_n

In [13]:
class weather_Dataset(Dataset):
    def __init__(self,X,Y):
        self.X = X
        self.Y = Y
    
    def __len__(self):
        return x_train.shape[0]
    
    def __getitem__(self,idx):
        return self.X[idx],self.Y[idx]

In [14]:
trainset = weather_Dataset(x_train,y_train)
validset = weather_Dataset(x_valid,y_valid)
testset = weather_Dataset(x_test,y_test)

In [16]:
#!pip install -U sentence-transformers==3.1.1
from sentence_transformers import SentenceTransformer

/Users/islandchen/opt/miniconda3/envs/dev_mode/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
model = SentenceTransformer('moka-ai/m3e-base')

In [ ]:
sentences = ["帐篷","手机","苹果手机"]

In [ ]:
from FlagEmbedding import FlagModel